In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

from config import facebook_account, facebook_password
from util import loginToFacebook, initBrowser, getPostLinks, getPostOrderData

In [2]:
from fbCrawler.models import (Product, Members, Orders, Helper, Session, Group, Checkout)
from fbCrawler.utils import *

import pandas as pd
import os, json,time, math,re,random,pickle,requests
from tqdm import tqdm

from datetime import datetime
from django.contrib.auth.models import User
from django.utils import timezone
from django.conf import settings
from django.utils.timezone import make_aware
from django.db.models import Q

# 流程

## 新增 Session

1. 新增 Session 物件，填入 session_id, group, 團購規則等資訊，有需要的話增加新的團購規則與社團資訊


## 結團爬蟲

1. 創建 allitems.csv 索引
2. 爬每篇文章的按讚會員，回傳 `[{name: 用戶 fb 名稱, id: 用戶 fbuid },{},{}...]` ，由此建立每個品項的按讚會員名單.csv 檔
3. 把 .csv 檔新增到 S3 做備份？

## 新增品項資料

1. 由 allitems.csv 建立品項物件，填入基本資料、session, year, month 等資訊


## 新增訂單

1. 新增會員資料
    1. 如果 fbuid 不存在，則建立新的會員資料，填入， FB Name, fburl, fbuid, group id 等資訊
    2. 分配小幫手
2. 建立訂單，預設數量為 0，填入 Session, member, product 等資訊


## 禁止抄襲爬蟲

1. 爬商品 id 與 po 文連結即可，更新到 Product 物件的 fburl

## 結算流程

1. 由小幫手提供的產品列表，更新最終價格、庫存、Erp 編號、限制購買數量、購買優惠等資訊 -> 上傳 .csv 參考套件：[import/export](https://django-import-export.readthedocs.io/en/latest/)
2. 購買權排序
    1. 買多的先，先不考慮買多排不到
    2. VIP 名單
    3. 其他 +1 的隨機
3. 購買權分配
4. 結算流程
5. 小幫手可以點擊開始結算程序

## 結算後大表單

1. 對帳資訊：
    1. 對帳沒有問題的資訊顯示為黃底，置頂
    2. 匯款金額錯誤的顯示為紅底
    3. 沒填單的預設白底
    4. 填寫出貨批次（text 欄位）
    5. 要顯示匯款時間（匯款時間早的排在前面）
    6. 對帳與金額外的資料，點擊用戶名稱顯示用戶基本資料頁面
    7. 用小幫手名稱排序
2. 上傳網銀明細.csv
    1. 對帳 `後五碼` 跟 `金額`
    * TODO: checkout model 增加後五碼、金額等資訊、最右邊出貨批次（數字）、應匯款金額（會因為自取或是宅配影響應付價格）
3. 網銀對帳單新建 table，增加 session 欄位
4. 發票：
    1. 黃色的填入我們這邊的資訊，其他的照格式留白
    2. 只有黃底（對帳無誤）的用戶才顯示

## 出貨表單

1. 顯示用戶資訊

## 測試

In [4]:
# browser = initBrowser()

# loginToFacebook(browser, facebook_account, facebook_password)

# time.sleep(5)
postDatas = getPostLinks(browser,
                        groupUrl = 'https://www.facebook.com/groups/TDa.BuyWine/?ref=bookmarks',
                        keywords = '2019/9月')

scrolling posts
clicking see more buttons
removing non-selling posts
processing all item's information


## 結團爬蟲

In [6]:
# 爬按讚名單
dataFolder = 'localDevDatas'

def InnoproFBLikeCrawler(groupUrl, keywords, groupName, year, month):
    startTime = time.time()
    # 設定 pandas col 顯示字數
    pd.set_option('max_colwidth',512)
    currentPath = currentPath = os.getcwd()
    
    if not os.path.exists(f"{currentPath}/{dataFolder}/"):
            os.mkdir(f"{currentPath}/{dataFolder}/")
    
    if not os.path.exists(f"{currentPath}/{dataFolder}/{groupName}"):
            os.mkdir(f"{currentPath}/{dataFolder}/{groupName}")
    
    if not os.path.exists(f"{currentPath}/{dataFolder}/{groupName}/{str(year)}{str(month)}"):
        os.mkdir(f"{currentPath}/{dataFolder}/{groupName}/{str(year)}{str(month)}")

    browser = initBrowser()
    
    loginToFacebook(browser, facebook_account, facebook_password)
    
    time.sleep(5)
    postDatas = getPostLinks(browser,
                            groupUrl = groupUrl,
                            keywords = keywords)
    
    postDatas.to_csv(f'{currentPath}/{dataFolder}/{groupName}/{str(year)}{str(month)}/{groupName}_{str(year)}{str(month)}_allitems.csv')
    print(f'post datas saved as {groupName}_{str(year)}{str(month)}_allitems.csv')
    
    print('processing each post content')
    time.sleep(0.5)
    
    postLinks = postDatas['link'].values.tolist()
    wineIds = postDatas['item_id'].values.tolist()
    pbar = tqdm(postLinks)

    for index, link in enumerate(pbar):
        
        pbar.set_description(f'processing: {index}th post content')
        try:
            df = getPostLikes(browser, link)
            wine_id = wineIds[index]
            df.to_csv(f'{currentPath}/{dataFolder}/{groupName}/{year}{month}/{groupName}_{year}{month}_{str(wine_id)}.csv')
    
        except Exception as e:
            print(f'processing {index}th post in postdatas')
            print(e)
        
        time.sleep(1)

    elapsed_time = time.time() - startTime
    elapsed_time_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(f'finished, cost {elapsed_time_str}.')
    browser.close()

In [7]:
# Todo: add user profile url!!!
def getPostLikes(browser,link):
    browser.get(link)
    df = pd.DataFrame(columns=['username','fbuid'])
    try:
        firstRecEl = browser.find_element_by_xpath(
            "//a[contains(@href,'/ufi/reaction/profile/browser/') and contains(@ajaxify,'/ufi/reaction/profile/dialog')]")
        url = firstRecEl.get_attribute('href')
        browser.get(url)
    except Exception as e:
        if NoSuchElementException:
            print('this post has no like')
            return df
        else:
            print(e)
    
    isLikesLoaded = False

    while(isLikesLoaded == False):
        try:
            # scroll to btn position
            seeMoreBtn = browser.find_element_by_xpath(
                "//a[contains(@href,'ufi/reaction/profile/browser/fetch/?limit=50&reaction_type=1')]")
            seeMoreBtnOffsetY = seeMoreBtn.location['y'] - 300
            browser.execute_script(
                "window.scrollTo(0, {});".format(seeMoreBtnOffsetY))
            seeMoreBtn.click()
            time.sleep(2)
        except Exception as e :
            if NoSuchElementException:
                isLikesLoaded = True
                print('All likes loaded')
            else:
                print(e)
                isLikesLoaded = True

    likeLis = browser.find_element_by_xpath("//ul[@id='reaction_profile_browser1']").find_elements_by_tag_name('li')

    for index, li in enumerate(likeLis):
        userATag = li.find_element_by_xpath(".//a[contains(@href,'facebook.com') and contains(@data-gt,'eng_tid')]")
        user_data = json.loads(userATag.get_attribute('data-gt'))
        fbuid = user_data['engagement']['eng_tid']
        fbname = userATag.text
        df.loc[index] = (fbname, fbuid)
    return df

## 初始化結團流程

In [8]:
session_id = '20190911'
groupUrl = 'https://www.facebook.com/groups/TDa.BuyWine/?ref=bookmarks'
keywords = '2019/9月'

In [9]:
InnoproFBLikeCrawler(groupUrl, keywords, 'Tda', '2019', '09')

logging in to fb
You already logged in
scrolling posts
clicking see more buttons
removing non-selling posts
processing all item's information
post datas saved as Tda_201909_allitems.csv
processing each post content


processing: 0th post content:   0%|          | 0/63 [00:00<?, ?it/s]

All likes loaded


processing: 1th post content:   2%|▏         | 1/63 [00:04<04:33,  4.42s/it]

All likes loaded


...

All likes loaded


processing: 62th post content:  98%|█████████▊| 62/63 [07:20<00:08,  8.27s/it]

All likes loaded


processing: 62th post content: 100%|██████████| 63/63 [07:50<00:00, 14.84s/it]

finished, cost 00:19:55.


## 新增品項資料

In [15]:
# load all items.csv
allitemsDF = pd.read_csv('localDevDatas/Tda/201909/Tda_201909_allitems.csv', index_col=0)

In [12]:
# allitemsDF

In [16]:
session = Session.objects.get(session_id = session_id)
for index,item in allitemsDF.iterrows():
    print(index)
    
    # check if product exist

    product_qs = Product.objects.filter(
        session = session, 
        wine_id = item.item_id
    )

    if(product_qs.count()==0):
        print('not exist')
        newPd = Product(
            session = session,
            wine_id = item.item_id,
            name = item.item_name,
            year = item.year,
            month = item.month,
            product_fb_link = item.link,
            price = item.price
        )
        newPd.save()
        print('created')
    else:
        print('existed')
        product_qs.update(
            name = item.item_name,
            year = item.year,
            month = item.month,
            product_fb_link = item.link,
            price = item.price
        )
        print('updated')

0
existed
updated
1
existed
updated
2
existed
updated
3
existed
updated
4
existed
updated
5
existed
updated
6
existed
updated
7
existed
updated
8
existed
updated
9
existed
updated
10
existed
updated
11
existed
updated
12
existed
updated
13
existed
updated
14
existed
updated
15
existed
updated
16
existed
updated
17
existed
updated
18
existed
updated
19
existed
updated
20
existed
updated
21
existed
updated
22
existed
updated
23
existed
updated
24
existed
updated
25
existed
updated
26
existed
updated
27
existed
updated
28
existed
updated
29
existed
updated
30
existed
updated
31
existed
updated
32
existed
updated
33
existed
updated
34
existed
updated
35
existed
updated
36
existed
updated
37
existed
updated
38
existed
updated
39
existed
updated
40
existed
updated
41
existed
updated
42
existed
updated
43
existed
updated
44
existed
updated
45
existed
updated
46
existed
updated
47
existed
updated
48
existed
updated
49
existed
updated
50
existed
updated
51
existed
updated
52
existed
updated
53


## 新增會員與訂單

# Todo: add user profile url!!! in like crawler and add member

In [17]:
# get each item list in folder
Tda_all_item_name = [
    file for file in os.listdir('localDevDatas/Tda/201909/') if 'Tda' in file and 'allitems' not in file]


In [19]:
len(Tda_all_item_name)

64

In [20]:
session = Session.objects.get(session_id = session_id)
pbar = tqdm(Tda_all_item_name)
for index, file_name in enumerate(pbar):
    pbar.set_description(f'processing: {index}th post content')    
    wine_id = file_name.split('_')[2].split('.')[0]
    product = Product.objects.get(wine_id = wine_id, session = session)
    
    print(wine_id)
    
    current_df = pd.read_csv(f'localDevDatas/Tda/201909/{file_name}', index_col=0)
    for index, memberinfo in current_df.iterrows():
        username = memberinfo.username
        fbuid = memberinfo.fbuid
        member_qs = Members.objects.filter(fbid = fbuid)
        print(fbuid)
        
        # create member
        if (member_qs.count() == 0):
            print('user not exist')
            member = Members(name = username, fbid = fbuid)
            member.save()
            print('user created')
        else:
            print('user already existed')
            member = member_qs[0]
        
        
        order_qs = Orders.objects.filter(session = session, product = product, member = member)

        # create order
        if (order_qs.count() == 0):
            print('order not exist')
            order = Orders(
                session = session,
                product = product,
                member = member,
                amount = 0
            )
            order.save()
            print('order created')
        else:
            print('order existed')
            order_qs.update(
                session = session,
                product = product,
                member = member,
                amount = 0
            )
            print('order updated')

processing: 0th post content:   0%|          | 0/64 [00:00<?, ?it/s]

4527
100002724731620
user already existed
order not exist
order created
1364929687
user already existed
order existed
order updated
100003670522210
user already existed
order existed
order updated
100000154546428
user already existed
order existed
order updated
100000217091240
user already existed
order existed
order updated
100001763721407
user already existed
order existed
order updated
100000136075797
user already existed
order existed
order updated
100037640469284
user already existed
order not exist
order created
100004000718056
user already existed
order not exist
order created
1362420370
user already existed
order existed
order updated
100001094771297
user already existed
order existed
order updated
100000215568195
user already existed
order existed
order updated
100003747695226
user already existed
order existed
order updated
100000489153985
user already existed
order existed
order updated
100000112875940
user already existed
order existed
order updated
100000692753097
user alr

processing: 1th post content:   2%|▏         | 1/64 [00:27<28:32, 27.18s/it]

order updated
4490
100039491349925
user already existed
order existed
order updated
100002724731620
user already existed
order existed
order updated
550452342
user already existed
order not exist
order created
100000154546428
user already existed
order existed
order updated
1680952070
user already existed
order existed
order updated
1419463751
user already existed
order not exist
order created
100037640469284
user already existed
order not exist
order created
100000606890690
user already existed
order not exist
order created
100001258093903
user already existed
order not exist
order created
100000029085283
user already existed
order not exist
order created
1302871681
user already existed
order not exist
order created
100004000718056
user already existed
order not exist
order created
1239478776
user already existed
order not exist
order created
100000117132858
user already existed
order existed
order updated
703574289
user already existed
order existed
order updated
1507980615
user alre

processing: 2th post content:   3%|▎         | 2/64 [01:05<31:41, 30.67s/it]

...

order created
4660
100000403492915
user already existed
order not exist
order created
100001838366628
user already existed
order not exist
order created
100022878082585
user already existed
order not exist
order created
100000160724344
user already existed
order not exist
order created
100000138424633
user already existed
order not exist
order created
1203113640
user already existed
order not exist
order created
100001371083147
user already existed
order not exist
order created
100000183955317
user already existed
order not exist
order created
1343780847
user already existed
order not exist
order created
1637738227
user already existed
order not exist
order created
100000318933462
user already existed
order not exist
order created
100000263102044
user already existed
order not exist
order created
100000058270349
user already existed
order not exist
order created
100000258482244
user already existed
order not exist
order created
100001111117735
user already existed
order not exist
order

processing: 63th post content:  98%|█████████▊| 63/64 [30:46<00:37, 37.28s/it]

order created
4502
1697332988
user already existed
order not exist
order created
100001838366628
user already existed
order not exist
order created
100022878082585
user already existed
order not exist
order created
100001677004076
user already existed
order not exist
order created
100000594210322
user already existed
order not exist
order created
100000207109009
user already existed
order not exist
order created
100000138424633
user already existed
order not exist
order created
100006050499595
user already existed
order not exist
order created
1069749398
user already existed
order not exist
order created
100000606890690
user already existed
order not exist
order created
1306239233
user already existed
order not exist
order created
36914827
user already existed
order not exist
order created
100000112875940
user already existed
order not exist
order created
567634140
user already existed
order not exist
order created
100000277834805
user already existed
order not exist
order created
1000

processing: 63th post content: 100%|██████████| 64/64 [31:06<00:00, 31.97s/it]

order created


## 分配小幫手

In [21]:
memberWithoutHelper = Members.objects.filter(helper = None)
helper1 = Helper.objects.get(id=1)
helper2 = Helper.objects.get(id=2)
helper3 = Helper.objects.get(id=3)
noHelperAmount = len(memberWithoutHelper)
print(noHelperAmount)

0


In [22]:
h1_amount = round(noHelperAmount*(helper1.work_percent/100))
h2_amount = round(noHelperAmount*(helper2.work_percent/100))
h3_amount = round(noHelperAmount*(helper3.work_percent/100))
# print(h1_amount,h2_amount,h3_amount, h1_amount+h2_amount+h3_amount)
helper1_mbs = memberWithoutHelper[0:h1_amount]
helper2_mbs = memberWithoutHelper[h1_amount:h1_amount+h2_amount]
helper3_mbs = memberWithoutHelper[h1_amount+h2_amount:]

In [23]:
print(helper1_mbs,helper2_mbs, helper3_mbs)

[] [] []


In [14]:
for mb in helper1_mbs:
    mb.helper = helper1
    mb.save()
    
for mb in helper2_mbs:
    mb.helper = helper2
    mb.save()
    
for mb in helper3_mbs:
    mb.helper = helper3
    mb.save()

## Update profile url

In [34]:
d = datetime(2019,9,10)

In [42]:
no_profile_user = Members.objects.filter(user_profile_url = None)

In [43]:
len(no_profile_user)

163

In [6]:
browser = initBrowser()

In [7]:
browser.get('file:///Users/ankycheng/Desktop/mb.html')

In [8]:
allmbs = browser.find_element_by_xpath("//ul[contains(@class, 'uiList clearfix _5bbv _4kg _704 _4ks')]")
mbs = allmbs.find_elements_by_xpath("//div[contains(@class, 'clearfix _60rh _gse')]")

In [10]:
len(mbs)

4930

In [11]:
mb_list = []

for index, mb in enumerate(mbs):
    mb_atag = mb.find_element_by_tag_name('a')
    
    try:
        # fburl
        href = mb_atag.get_attribute('href')
        if ('?fref' in href):
            href = href.split('?fref')[0]
        elif ('&fref=' in href):
            href = href.split('&fref=')[0]
        else:
            print('error', index)
        fb_url = href

        fb_name = mb_atag.get_attribute('title')
        
        pf_data = mb_atag.get_attribute('data-hovercard')
        start = pf_data.find('php?id=')+7
        end = pf_data.find('&')
        fbid = pf_data[start:end]

        mb = {
            'fbid': fbid,
            'fburl': fb_url,
            'fbname': fb_name
        }

        mb_list.append(mb)
    except Exception as e:
        print('error', index)
        print(e)

## 新增結算結果 （Orders and Checkouts）

In [3]:
finalOrders = pd.read_csv('localDevDatas/Tda/201909/checkout/afterSettleList.csv', index_col=0)
checkouts = pd.read_csv('localDevDatas/Tda/201909/checkout/feeList.csv', index_col=0)

In [8]:
# finalOrders

In [4]:
# checkouts

In [7]:
me = Members.objects.get(id=1)
me

<Members: 鄭安凱 100000086225914>

## Update checkouts(Feelist)

In [4]:
session_id = '20190911'
session = Session.objects.get(session_id = session_id)

In [11]:
c0 = checkouts.iloc[0]
c0

session         20190911
fbuid           28121540
訂單編號        201909110001
結算確認購買數量               1
金額達標免運優惠               無
瓶數達標免運優惠               無
全團折扣優惠           無全團折扣優惠
折扣前價格                580
折扣後自取價               580
運費                   160
含運費總價                740
Name: 0, dtype: object

In [ ]:
c0

In [13]:


c = Checkout(
        member = me,
        session = session,
        checkout_id = 201909119999,
        price_before_disc = 580,
        price_after_disc = 580,
        ship_fee = 160,
        total_price = 740
)
c.save()

In [13]:
for (index,checkout) in checkouts.iterrows():
    member = Members.objects.get(fbid = checkout['fbuid'])
    
    checkout_id = checkout['訂單編號']
    
    price_before_disc = checkout['折扣前價格']
    price_after_disc = checkout['折扣後自取價']
    ship_fee = checkout['運費']
    total_price = checkout['含運費總價']
    print(f'index: {index}, {member}, {checkout_id}, \n {price_before_disc}, {price_after_disc}, \n {ship_fee}, {total_price}')
    
    # check if checkout exist
    checkout_qs = Checkout.objects.filter(member = member, session = session)
    if len(checkout_qs) == 0:
        
        print('checkout not existed, creating...')
        checkout = Checkout(
            member = member,
            session = session,
            checkout_id = checkout_id,
            price_before_disc = price_before_disc,
            price_after_disc = price_after_disc,
            ship_fee = ship_fee,
            total_price = total_price
        )
        checkout.save()
    else:
        print('checkout existed, updating... ')
        checkout_qs.update(
            checkout_id = checkout_id,
            price_before_disc = price_before_disc,
            price_after_disc = price_after_disc,
            ship_fee = ship_fee,
            total_price = total_price
        )

index: 0, Jeremie Pan 28121540, 201909110001, 
 580, 580, 
 160, 740
checkout not existed, creating...
index: 1, Vincent Tk Kuo 36914827, 201909110002, 
 13640, 13640, 
 290, 13930
checkout not existed, creating...
index: 2, Sean Sung 518125422, 201909110003, 
 750, 750, 
 160, 910
checkout not existed, creating...
index: 3, Yen-Shao Kuo 519998038, 201909110004, 
 1080, 1080, 
 160, 1240
checkout not existed, creating...
index: 4, 安剔特 524402757, 201909110005, 
 1680, 1680, 
 160, 1840
checkout not existed, creating...
index: 5, 薛博駿 525966570, 201909110006, 
 1690, 1690, 
 160, 1850
checkout not existed, creating...
index: 6, Feng-Bin Chang 526497993, 201909110007, 
 1320, 1320, 
 225, 1545
checkout not existed, creating...
index: 7, Chang Lin 528043515, 201909110008, 
 950, 950, 
 160, 1110
checkout not existed, creating...
index: 8, Eon Chen 532934982, 201909110009, 
 750, 750, 
 160, 910
checkout not existed, creating...
index: 9, Charles Wu 544413512, 201909110010, 
 3780, 3780, 
 2

index: 79, 陳哲民 773483329, 201909110080, 
 4870, 4870, 
 225, 5095
checkout not existed, creating...
index: 80, Mel Lu 801975100, 201909110081, 
 680, 680, 
 160, 840
checkout not existed, creating...
index: 81, Helen Wu 805757444, 201909110082, 
 1570, 1570, 
 225, 1795
checkout not existed, creating...
index: 82, Jared Yu 808454207, 201909110083, 
 6210, 6210, 
 225, 6435
checkout not existed, creating...
index: 83, 林賽門 878175461, 201909110084, 
 800, 800, 
 160, 960
checkout not existed, creating...
index: 84, 李富偉 1004051352, 201909110085, 
 10950, 10950, 
 225, 11175
checkout not existed, creating...
index: 85, Amy Hsiao 1008339261, 201909110086, 
 680, 680, 
 160, 840
checkout not existed, creating...
index: 86, Stanley Ko 1011165559, 201909110087, 
 9550, 9550, 
 225, 9775
checkout not existed, creating...
index: 87, 21 1027668577, 201909110088, 
 1760, 1760, 
 225, 1985
checkout not existed, creating...
index: 88, Alex Chen 1037568251, 201909110089, 
 1690, 1690, 
 225, 1915
chec

index: 156, Nils Hsu 1511497632, 201909110157, 
 580, 580, 
 160, 740
checkout not existed, creating...
index: 157, Frank Fang 1520874579, 201909110158, 
 15730, 15730, 
 515, 16245
checkout not existed, creating...
index: 158, Terry Huang 1521807497, 201909110159, 
 3620, 3620, 
 225, 3845
checkout not existed, creating...
index: 159, 李孝偉 1524387869, 201909110160, 
 10760, 10760, 
 225, 10985
checkout not existed, creating...
index: 160, Wei-Chieh Fang 1530346867, 201909110161, 
 1160, 1160, 
 225, 1385
checkout not existed, creating...
index: 161, John John Wu 1541151508, 201909110162, 
 3530, 3530, 
 225, 3755
checkout not existed, creating...
index: 162, Charles Hung 1543209928, 201909110163, 
 9650, 9650, 
 225, 9875
checkout not existed, creating...
index: 163, Barkiel Huang 1544277274, 201909110164, 
 11550, 11550, 
 290, 11840
checkout not existed, creating...
index: 164, Hsuan-Chih Kuo 1559790794, 201909110165, 
 2710, 2710, 
 225, 2935
checkout not existed, creating...
index:

checkout not existed, creating...
index: 232, Jack Jiang 100000100410528, 201909110233, 
 1330, 1330, 
 225, 1555
checkout not existed, creating...
index: 233, 魏辰澔 100000112875940, 201909110234, 
 14990, 14990, 
 290, 15280
checkout not existed, creating...
index: 234, 黃貞凱 100000113425797, 201909110235, 
 7120, 7120, 
 225, 7345
checkout not existed, creating...
index: 235, Stone Lee 100000116057371, 201909110236, 
 2540, 2540, 
 225, 2765
checkout not existed, creating...
index: 236, Rex Lee 100000117132858, 201909110237, 
 9330, 9330, 
 290, 9620
checkout not existed, creating...
index: 237, 溫盛宇 100000118839096, 201909110238, 
 750, 750, 
 160, 910
checkout not existed, creating...
index: 238, Derek Lee 100000119508630, 201909110239, 
 1350, 1350, 
 225, 1575
checkout not existed, creating...
index: 239, Calvin Yuan 100000120064116, 201909110240, 
 580, 580, 
 160, 740
checkout not existed, creating...
index: 240, 林伊森 100000121902667, 201909110241, 
 3650, 3650, 
 225, 3875
checkout 

index: 306, 陳瀚翔 100000302659885, 201909110307, 
 5360, 5360, 
 225, 5585
checkout not existed, creating...
index: 307, 鄭淑玲 100000318933462, 201909110308, 
 2080, 2080, 
 225, 2305
checkout not existed, creating...
index: 308, 顧春瑀 100000322003770, 201909110309, 
 1480, 1480, 
 160, 1640
checkout not existed, creating...
index: 309, Ingrid Huang 100000325652535, 201909110310, 
 1480, 1480, 
 225, 1705
checkout not existed, creating...
index: 310, Yi-Hung Pan 100000332146403, 201909110311, 
 5980, 5980, 
 225, 6205
checkout not existed, creating...
index: 311, Joanna Wen 100000335738811, 201909110312, 
 1780, 1780, 
 160, 1940
checkout not existed, creating...
index: 312, 楊鈞逸 100000349449040, 201909110313, 
 4440, 4440, 
 225, 4665
checkout not existed, creating...
index: 313, Jacko Wu 100000350442020, 201909110314, 
 10960, 10960, 
 290, 11250
checkout not existed, creating...
index: 314, Awu Lin 100000350471087, 201909110315, 
 3920, 3920, 
 225, 4145
checkout not existed, creating...
i

index: 380, Chia Yi Chuang 100001647572620, 201909110381, 
 2930, 2930, 
 225, 3155
checkout not existed, creating...
index: 381, Mercy Chang 100001677004076, 201909110382, 
 7720, 7720, 
 290, 8010
checkout not existed, creating...
index: 382, Egg Lu 100001724937838, 201909110383, 
 20950, 20950, 
 515, 21465
checkout not existed, creating...
index: 383, Chi-Chih Hung 100001763721407, 201909110384, 
 37010, 37010, 
 515, 37525
checkout not existed, creating...
index: 384, 鍾大衛 100001805362776, 201909110385, 
 6600, 6600, 
 225, 6825
checkout not existed, creating...
index: 385, 童加菲 100001835799590, 201909110386, 
 1780, 1780, 
 160, 1940
checkout not existed, creating...
index: 386, Rudy Cheng 100001838366628, 201909110387, 
 14960, 14960, 
 290, 15250
checkout not existed, creating...
index: 387, Joyce Wei 100001846786695, 201909110388, 
 4410, 4410, 
 225, 4635
checkout not existed, creating...
index: 388, Riot Young 100001848033496, 201909110389, 
 2380, 2380, 
 225, 2605
checkout n

index: 454, 陳俊宏 100028715590502, 201909110455, 
 750, 750, 
 160, 910
checkout not existed, creating...
index: 455, Lydia Zhang 100034722640068, 201909110456, 
 1430, 1430, 
 225, 1655
checkout not existed, creating...
index: 456, Lord Redwyne 100037640469284, 201909110457, 
 10090, 10090, 
 290, 10380
checkout not existed, creating...


In [14]:
finalOrders

,session,fbuid,FB姓名,購買權確認,商品編號,商品名稱,購買限制,留言購買數,結算確認購買數量,優惠贈送量,扣除贈送量後計價數,商品單價,商品總價,備註
0,20190911,100000090961284,洪思群,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
1,20190911,643760847,Bai Xiao Bai,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
2,20190911,1520874579,Frank Fang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
3,20190911,580856401,Herb Chang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
4,20190911,100001106450487,曾彥文,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
5,20190911,100007086393208,Da Chen Pang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
6,20190911,100002296332595,Carlos Hsieh,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
7,20190911,100000090961284,洪思群,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
8,20190911,100000138424633,YH Chang,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認
9,20190911,100004003127767,蔡宗霖,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認


## Update FinalOrders(afterSettleList)

In [9]:
for (index,finalOrder) in finalOrders.iterrows():

    member = Members.objects.get(fbid = finalOrder['fbuid'])
    print(f'{index}: {member}')
    product = Product.objects.get(wine_id = finalOrder['商品編號'], session = session)
    is_confrimed = True if finalOrder['商品編號'] == 'get' else False
    confirm_amount = finalOrder['結算確認購買數量']#改這樣:扣除贈送量後計價數
    #留言購買數也要加
    free_amount = finalOrder['優惠贈送量']
    minus_free_amount = finalOrder['扣除贈送量後計價數']#改這樣:結算確認購買數量
    notes = finalOrder['備註']

    order_qs = Orders.objects.filter(member = member, session = session, product = product)
    # print(order_qs)
    order_qs.update(
        confirm_amount = confirm_amount,
        free_amount = free_amount,
        minus_free_amount = minus_free_amount,
        is_confrimed = is_confrimed,
        notes = notes
    )

0: 洪思群 100000090961284
<QuerySet [<Orders: 洪思群  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
1: Bai Xiao Bai 643760847
<QuerySet [<Orders: Bai Xiao Bai  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
2: Frank Fang 1520874579
<QuerySet [<Orders: Frank Fang  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
3: Herb Chang 580856401
<QuerySet [<Orders: Herb Chang  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
4: 曾彥文 100001106450487
<QuerySet [<Orders: 曾彥文  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
5: Da Chen Pang 100007086393208
<QuerySet [<Orders: Da Chen Pang  Benedicte et Stephane Tissot 2016 Arbois Chardonnay Les Graviers  20190911>]>
6: Carlos Hsieh 100002296332595
<QuerySet [<Orders: Carlos Hsieh  Benedicte et Stephane Tissot 2017 Arbois DD Rouge  20190911>]>
7: 洪思群 100000090961284
<QuerySet [<Orders: 洪思群  Benedicte et Stephan

67: Gary Chang 100002724731620
<QuerySet [<Orders: Gary Chang  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
68: Rudy Cheng 100001838366628
<QuerySet [<Orders: Rudy Cheng  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
69: Nelson Fang 1439769770
<QuerySet [<Orders: Nelson Fang  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
70: Jeremy Chen 1765084847
<QuerySet [<Orders: Jeremy Chen  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
71: 郭福三 100000207109009
<QuerySet [<Orders: 郭福三  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
72: Ping Hsuan Kao 100000560511952
<QuerySet [<Orders: Ping Hsuan Kao  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
73: Joanna Wen 100000335738811
<QuerySet [<Orders: Joanna Wen  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
74: Chi-Chih Hung 100001763721407
<QuerySet [<Orders: Chi-Chih Hung  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
75: Leiwai Chau 100000205300467
<QuerySet [<Orders: Leiwai Chau  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
76: K

<QuerySet [<Orders: Joseph Hsu  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
143: Bernice Lin 1074545854
<QuerySet [<Orders: Bernice Lin  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
144: 施翔筌 100002699309204
<QuerySet [<Orders: 施翔筌  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
145: 溫盛宇 100000118839096
<QuerySet [<Orders: 溫盛宇  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
146: Chu Chu Hsieh 100000557497102
<QuerySet [<Orders: Chu Chu Hsieh  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
147: Roy Lo 1648338633
<QuerySet [<Orders: Roy Lo  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
148: Gary Chang 100002724731620
<QuerySet [<Orders: Gary Chang  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
149: Ping Hsuan Kao 100000560511952
<QuerySet [<Orders: Ping Hsuan Kao  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
150: 王永興 100016231086008
<QuerySet [<Orders: 王永興  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
151: 魏辰澔 100000112875940
<QuerySet [<Ord

217: Stanley Ko 1011165559
<QuerySet [<Orders: Stanley Ko  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
218: 安剔特 524402757
<QuerySet [<Orders: 安剔特  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
219: Austin Ko 100006050499595
<QuerySet [<Orders: Austin Ko  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
220: Lord Redwyne 100037640469284
<QuerySet [<Orders: Lord Redwyne  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
221: Matt Yeh 1576784388
<QuerySet [<Orders: Matt Yeh  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
222: 李宏祥 1288033764
<QuerySet [<Orders: 李宏祥  Tenuta San Guido 2015 le Difese紅 20190911>]>
223: 林申昌 100026354479057
<QuerySet [<Orders: 林申昌  Tenuta San Guido 2015 le Difese紅 20190911>]>
224: Leon Chen 1847646753
<QuerySet [<Orders: Leon Chen  Tenuta San Guido 2015 le Difese紅 20190911>]>
225: Carol Wu 100004278498557
<QuerySet [<Orders: Carol Wu  Tenuta San Guido 2015 le Difese紅 20190911>]>
226: Andy Yu 100002606253012
<QuerySet [<Orders: Andy Yu  Tenuta 

<QuerySet [<Orders: Minshan Tseng  Tenuta San Guido 2015 le Difese紅 20190911>]>
297: 秦一男 100000179515951
<QuerySet [<Orders: 秦一男  Tenuta San Guido 2015 le Difese紅 20190911>]>
298: Yell G Yao 609707335
<QuerySet [<Orders: Yell G Yao  Tenuta San Guido 2015 le Difese紅 20190911>]>
299: Bing-Yen Lee 100000204807065
<QuerySet [<Orders: Bing-Yen Lee  Tenuta San Guido 2015 le Difese紅 20190911>]>
300: Wilber Jheng 1729157572
<QuerySet [<Orders: Wilber Jheng  Tenuta San Guido 2015 le Difese紅 20190911>]>
301: Vivian Lo 1096970679
<QuerySet [<Orders: Vivian Lo  Tenuta San Guido 2015 le Difese紅 20190911>]>
302: Myron Lu 571503629
<QuerySet [<Orders: Myron Lu  Tenuta San Guido 2015 le Difese紅 20190911>]>
303: Katy Chou 632586282
<QuerySet [<Orders: Katy Chou  Tenuta San Guido 2015 le Difese紅 20190911>]>
304: Fanny Hsu 744038431
<QuerySet [<Orders: Fanny Hsu  Tenuta San Guido 2015 le Difese紅 20190911>]>
305: Brian Huang 1680952070
<QuerySet [<Orders: Brian Huang  Tenuta San Guido 2015 le Difese紅 2019

...

<QuerySet [<Orders: AJ Wang  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1547: Alex Lin 100000235540485
<QuerySet [<Orders: Alex Lin  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1548: Kevin Li 1058681632
<QuerySet [<Orders: Kevin Li  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1549: Evan Chou 100000891657095
<QuerySet [<Orders: Evan Chou  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1550: 陳哲民 773483329
<QuerySet [<Orders: 陳哲民  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1551: Adam Cheng 560410780
<QuerySet [<Orders: Adam Cheng  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1552: Kevin Ko 649769832
<QuerySet [<Orders: Kevin Ko  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1553: Tessa Chiang 665821576
<QuerySet [<Orders: Tessa Chiang  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1554: 莊承倫 1690639940
<QuerySet [<Orders: 莊承倫  Ayala 2008 Rosé No.8 Brut Champagne 20190911>]>
1555: Eugene Liu 702502143
<QuerySet [<Orders: Eugene Liu  Ayala 2008 Rosé No.8 Brut Champa

1623: Wilber Jheng 1729157572
<QuerySet [<Orders: Wilber Jheng  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1624: 李明璋 1203113640
<QuerySet [<Orders: 李明璋  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1625: 鄭淑玲 100000318933462
<QuerySet [<Orders: 鄭淑玲  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1626: Charling Kung 100000178981285
<QuerySet [<Orders: Charling Kung  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1627: Joanne Liang 652319344
<QuerySet [<Orders: Joanne Liang  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1628: Petitbtl Ang 100022878082585
<QuerySet [<Orders: Petitbtl Ang  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1629: 彭宜庭 100005406932446
<QuerySet [<Orders: 彭宜庭  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1630: Harry Ju 559407203
<QuerySet [<Orders: Harry Ju  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>]>
1631: Po-Yen Lin 100001371083147
<QuerySet [<Orders: Po-Yen Lin  Dominique Laurent 2015 Marsannay V.V. 紅 20190911>

1697: Austin Ko 100006050499595
<QuerySet [<Orders: Austin Ko  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1698: Brian Huang 1680952070
<QuerySet [<Orders: Brian Huang  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1699: Lord Redwyne 100037640469284
<QuerySet [<Orders: Lord Redwyne  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1700: 葉春盛 100006376187566
<QuerySet [<Orders: 葉春盛  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1701: Chun Tai Tsai 1239478776
<QuerySet [<Orders: Chun Tai Tsai  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1702: Ming Yang 1151941678
<QuerySet [<Orders: Ming Yang  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1703: Marvin Liang 1047741629
<QuerySet [<Orders: Marvin Liang  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1704: Eric Liu 650622012
<QuerySet [<Orders: Eric Liu  Dumenil 2008 Brut 1er Cru Champagne香檳 20190911>]>
1705: Chia Yi Chuang 100001647572620
<QuerySet [<Orders: Chia Yi Chuang  Dumenil 2008 Brut 1er Cru Champagne香檳 2

<QuerySet [<Orders: 莊智宇  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1768: Catherine Chang 1288080998
<QuerySet [<Orders: Catherine Chang  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1769: Honda Chou 762274564
<QuerySet [<Orders: Honda Chou  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1770: 周正欣 575194747
<QuerySet [<Orders: 周正欣  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1771: Joanne Chien 100000199934609
<QuerySet [<Orders: Joanne Chien  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1772: Nick Wang 100000058270349
<QuerySet [<Orders: Nick Wang  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1773: Yu-Chao Chen 100000277834805
<QuerySet [<Orders: Yu-Chao Chen  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1774: 鍾耀璇 1433024461
<QuerySet [<Orders: 鍾耀璇  Albert Bichot 2015 Bourgogne Vieilles Vignes紅 20190911>]>
1775: Jessy Chen 730102949
<QuerySet [<Orders: Jessy Chen  Albert Bichot 2015 Bourgogne Vi

1837: Wei Chung Su 100000242022191
<QuerySet [<Orders: Wei Chung Su  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1838: Ben Tsai 100000415902078
<QuerySet [<Orders: Ben Tsai  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1839: Po-chih Tsai 100001013239357
<QuerySet [<Orders: Po-chih Tsai  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1840: 李修明 100002250183167
<QuerySet [<Orders: 李修明  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1841: Ryan Huang 1503133340
<QuerySet [<Orders: Ryan Huang  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1842: Hsuan-Chih Kuo 1559790794
<QuerySet [<Orders: Hsuan-Chih Kuo  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1843: 301 100000140272310
<QuerySet [<Orders: 301  Trimbach 2007 Riesling Cuvee Frederic Emile白 20190911>]>
1844: Jason Lee 100000135060914
<QuerySet [<Orders: Jason Lee  Digioia Royer 2015 Chambolle Musigny 紅 20190911>]>
1845: Grace Hung 661206885
<QuerySet [<Orders: Grace Hung 

In [10]:
Co = Checkout.objects.get(id=4)

In [11]:
Co

<Checkout: 20190911 Jeremie Pan>

In [12]:
Co.payable_amount()

740

In [15]:
finalOrders

,session,fbuid,FB姓名,購買權確認,商品編號,商品名稱,購買限制,留言購買數,結算確認購買數量,優惠贈送量,扣除贈送量後計價數,商品單價,商品總價,備註
0,20190911,100000090961284,洪思群,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
1,20190911,643760847,Bai Xiao Bai,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
2,20190911,1520874579,Frank Fang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
3,20190911,580856401,Herb Chang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
4,20190911,100001106450487,曾彥文,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
5,20190911,100007086393208,Da Chen Pang,get,4609,Benedicte et Stephane Tissot 2016 Arbois Chard...,無限加限制,1,1,0,1,1250,1250,已確認
6,20190911,100002296332595,Carlos Hsieh,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
7,20190911,100000090961284,洪思群,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,1,1,0,1,950,950,已確認
8,20190911,100000138424633,YH Chang,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認
9,20190911,100004003127767,蔡宗霖,get,4610,Benedicte et Stephane Tissot 2017 Arbois DD Rouge,無限加限制,2,2,0,2,950,1900,已確認


In [14]:
checkouts

,session,fbuid,訂單編號,結算確認購買數量,金額達標免運優惠,瓶數達標免運優惠,全團折扣優惠,折扣前價格,折扣後自取價,運費,含運費總價
0,20190911,28121540,201909110001,1,無,無,無全團折扣優惠,580,580,160,740
1,20190911,36914827,201909110002,11,無,無,無全團折扣優惠,13640,13640,290,13930
2,20190911,518125422,201909110003,1,無,無,無全團折扣優惠,750,750,160,910
3,20190911,519998038,201909110004,1,無,無,無全團折扣優惠,1080,1080,160,1240
4,20190911,524402757,201909110005,1,無,無,無全團折扣優惠,1680,1680,160,1840
5,20190911,525966570,201909110006,1,無,無,無全團折扣優惠,1690,1690,160,1850
6,20190911,526497993,201909110007,2,無,無,無全團折扣優惠,1320,1320,225,1545
7,20190911,528043515,201909110008,1,無,無,無全團折扣優惠,950,950,160,1110
8,20190911,532934982,201909110009,1,無,無,無全團折扣優惠,750,750,160,910
9,20190911,544413512,201909110010,4,無,無,無全團折扣優惠,3780,3780,225,4005
